In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, matthews_corrcoef

In [2]:
import pandas as pd
df = pd.read_csv('dcorr_filtered_BBB_0.25.csv', low_memory=False)

In [6]:
df.head()

,SMILES,TopoPSA,ATSC1c,TopoPSA(NO),ATSC0c,nHetero,MID_h,nHBDon,PEOE_VSA10,nHBAcc,...,GATS2p,VE1_Dzse,VE1_Dzpe,ATS2Z,VE1_Dzare,VE1_Dzi,VE1_DzZ,VE1_Dzm,ZMIC0,BBB_Class
0,O=C(O)c1cc(N=Nc2ccc(S(=O)(=O)Nc3ccccn3)cc2)ccc1O,149.69,-0.869178,141.31,1.320560,10.0,19.060046,3.0,17.130826,7.0,...,1.079616,5.212883,5.213325,2100.0,5.213752,5.212634,5.207067,5.207055,89.697096,0
1,COC1(NC(=O)C(C(=O)O)c2ccc(O)cc2)C(=O)N2C(C(=O)...,231.60,-1.480445,206.30,2.406565,16.0,30.633874,4.0,11.446551,12.0,...,1.041694,5.905613,5.904109,2843.0,5.902724,5.902382,5.914851,5.914866,120.950183,0
2,Oc1c(I)cc(Cl)c2cccnc12,33.12,-0.246240,33.12,0.481581,4.0,7.337494,1.0,5.516701,2.0,...,0.591202,3.552646,3.553077,1513.0,3.548154,3.548148,3.571093,3.571244,49.257975,0
3,CCNC(=NCCSCc1ncccc1Br)NC#N,98.40,-0.406882,73.10,0.568644,7.0,13.439480,2.0,0.000000,4.0,...,1.069291,4.305903,4.307516,1568.0,4.308483,4.310238,4.293615,4.293574,67.893426,0
4,CN1CC[C@]23c4c5ccc(OC6O[C@H](C(=O)O)[C@@H](O)[...,149.15,-1.098008,149.15,2.216727,10.0,18.953495,5.0,30.519832,9.0,...,1.064031,5.660753,5.661139,2868.0,5.661487,5.662368,5.660800,5.660796,117.423288,0


In [9]:
X_desc = df.drop(columns=['SMILES', 'BBB_Class'])
y = df['BBB_Class']

In [11]:
X_desc.head(1)

,TopoPSA,ATSC1c,TopoPSA(NO),ATSC0c,nHetero,MID_h,nHBDon,PEOE_VSA10,nHBAcc,ATSC1se,...,GATS1c,GATS2p,VE1_Dzse,VE1_Dzpe,ATS2Z,VE1_Dzare,VE1_Dzi,VE1_DzZ,VE1_Dzm,ZMIC0
0,149.69,-0.869178,141.31,1.32056,10.0,19.060046,3.0,17.130826,7.0,-0.138205,...,1.547752,1.079616,5.212883,5.213325,2100.0,5.213752,5.212634,5.207067,5.207055,89.697096


In [13]:
import numpy as np
def remove_highly_correlated_features(df, threshold=0.85, method='spearman'):

    corr_matrix = df.corr(method=method).abs()

    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

    df_reduced = df.drop(columns=to_drop)

    return df_reduced, to_drop

In [15]:
X_uncorrelated, dropped_features = remove_highly_correlated_features(X_desc, threshold=0.85, method='spearman')
print("Dropped features:", dropped_features)

Dropped features: ['ATSC1c', 'TopoPSA(NO)', 'ATSC0c', 'nHetero', 'MID_h', 'nHBAcc', 'SM1_DzZ', 'AATS5se', 'n4AHRing', 'AATS5are', 'ATSC1are', 'AATS1se', 'SM1_Dzm', 'SM1_Dzse', 'AATS6are', 'AATS6se', 'AATS5pe', 'SM1_Dzv', 'AMID_C', 'AMID_h', 'AATS6pe', 'SM1_Dzare', 'SM1_Dzpe', 'AATS7are', 'AATS7pe', 'ATSC1pe', 'AATS1are', 'SM1_Dzi', 'n4Ring', 'n4ARing', 'AATS8dv', 'AATS1pe', 'AATS6dv', 'AATS7se', 'AATS8are', 'nO', 'AATS8pe', 'MID_O', 'AATS8se', 'AATS5dv', 'AATS7s', 'AATS1s', 'AATS6s', 'AATS3se', 'AATS4se', 'AATS7m', 'AATS0se', 'AATS2se', 'ATS0s', 'AATS8Z', 'SM1_Dzp', 'AATS8m', 'SMR_VSA1', 'AATS3are', 'AATS0are', 'IC0', 'AATS5Z', 'AATS4are', 'AATS6Z', 'AMID_O', 'AATS3s', 'AATS3pe', 'AATS4s', 'AATS5m', 'AATS0pe', 'ATS7dv', 'AATS6m', 'ATS0dv', 'AATS2are', 'MIC5', 'ETA_eta_F', 'AATS4pe', 'AATS2pe', 'ATSC0s', 'AATS2s', 'MIC4', 'EState_VSA1', 'GGI9', 'ATS8m', 'AATS7v', 'mZagreb1', 'NsOH', 'MIC3', 'SpMAD_Dzv', 'AATS0s', 'AATSC1s', 'GGI10', 'AATS4dv', 'ATS0Z', 'ATS1s', 'ATSC0se', 'SpMAD_D', 'AT

In [17]:
print(f"Original features (after variance filter): {X_desc.shape[1]}")
print(f"Remaining features (after correlation filter): {X_uncorrelated.shape[1]}")
print(f"Features removed due to high correlation: {len(dropped_features)}")

Original features (after variance filter): 292
Remaining features (after correlation filter): 59
Features removed due to high correlation: 233


In [19]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_uncorrelated, y, test_size=0.2, random_state=42, stratify=y)

In [21]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [25]:
# Convert to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

In [27]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [29]:
class MLPModel(nn.Module):
    def __init__(self, input_dim):
        super(MLPModel, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.fc(x)

model = MLPModel(input_dim=X_train.shape[1])

In [31]:
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 50
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(train_loader):.4f}")

Epoch 1/50, Loss: 0.4629
Epoch 2/50, Loss: 0.3961
Epoch 3/50, Loss: 0.3774
Epoch 4/50, Loss: 0.3625
Epoch 5/50, Loss: 0.3543
Epoch 6/50, Loss: 0.3463
Epoch 7/50, Loss: 0.3343
Epoch 8/50, Loss: 0.3325
Epoch 9/50, Loss: 0.3314
Epoch 10/50, Loss: 0.3220
Epoch 11/50, Loss: 0.3133
Epoch 12/50, Loss: 0.3097
Epoch 13/50, Loss: 0.3061
Epoch 14/50, Loss: 0.3004
Epoch 15/50, Loss: 0.2950
Epoch 16/50, Loss: 0.2910
Epoch 17/50, Loss: 0.2909
Epoch 18/50, Loss: 0.2828
Epoch 19/50, Loss: 0.2806
Epoch 20/50, Loss: 0.2774
Epoch 21/50, Loss: 0.2716
Epoch 22/50, Loss: 0.2661
Epoch 23/50, Loss: 0.2649
Epoch 24/50, Loss: 0.2622
Epoch 25/50, Loss: 0.2542
Epoch 26/50, Loss: 0.2582
Epoch 27/50, Loss: 0.2519
Epoch 28/50, Loss: 0.2502
Epoch 29/50, Loss: 0.2504
Epoch 30/50, Loss: 0.2460
Epoch 31/50, Loss: 0.2435
Epoch 32/50, Loss: 0.2359
Epoch 33/50, Loss: 0.2407
Epoch 34/50, Loss: 0.2341
Epoch 35/50, Loss: 0.2323
Epoch 36/50, Loss: 0.2364
Epoch 37/50, Loss: 0.2274
Epoch 38/50, Loss: 0.2365
Epoch 39/50, Loss: 0.

In [32]:
model.eval()
with torch.no_grad():
    y_pred_proba = model(X_test_tensor).numpy()
    y_pred_class = (y_pred_proba > 0.5).astype(int)

acc = accuracy_score(y_test, y_pred_class)
auc = roc_auc_score(y_test, y_pred_proba)
mcc = matthews_corrcoef(y_test, y_pred_class)

print(f"Accuracy: {acc:.4f}")
print(f"AUC: {auc:.4f}")
print(f"MCC: {mcc:.4f}")

Accuracy: 0.8815
AUC: 0.9428
MCC: 0.7443


### Applying GCN

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import matthews_corrcoef, roc_auc_score, accuracy_score, f1_score
from rdkit import Chem

C:\Users\vivek\anaconda3\envs\pytorch-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
df["label"] = df["BBB_Class"]

In [26]:
# SMILES → Graph

def mol_to_graph(smiles, label):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    
    # Node features: atom number only for now
    atom_features = []
    for atom in mol.GetAtoms():
        atom_features.append([atom.GetAtomicNum()])
    x = torch.tensor(atom_features, dtype=torch.float)

    # Edges
    edge_index = []
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        edge_index.append([i, j])
        edge_index.append([j, i])
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

    y = torch.tensor([label], dtype=torch.float)
    return Data(x=x, edge_index=edge_index, y=y)

In [28]:
# Create valid graph list
valid_graphs = []
valid_labels = []
for smiles, label in zip(df["SMILES"], df["label"]):
    graph = mol_to_graph(smiles, label)
    if graph:
        valid_graphs.append(graph)
        valid_labels.append(label)

valid_labels = torch.tensor(valid_labels, dtype=torch.long)

[08:21:58] Explicit valence for atom # 10 C, 4, is greater than permitted
[08:22:01] Explicit valence for atom # 10 C, 4, is greater than permitted


In [30]:
train_idx, test_idx = train_test_split(range(len(valid_graphs)),test_size=0.2,stratify=valid_labels,random_state=42)

train_loader = DataLoader([valid_graphs[i] for i in train_idx], batch_size=32, shuffle=True)
test_loader = DataLoader([valid_graphs[i] for i in test_idx], batch_size=32)

C:\Users\vivek\anaconda3\envs\pytorch-env\lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [32]:
# GCN model
class GCN(nn.Module):
    def __init__(self, num_node_features):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, 64)
        self.conv2 = GCNConv(64, 64)
        self.fc = nn.Linear(64, 1)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)  # Pooling
        x = self.fc(x)
        return torch.sigmoid(x)

In [34]:
# Training loop
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = GCN(num_node_features=1).to(device)  # Atom number only
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()

for epoch in range(30):
    model.train()
    total_loss = 0
    for batch in train_loader:
        batch = batch.to(device)
        optimizer.zero_grad()
        out = model(batch.x, batch.edge_index, batch.batch)
        loss = criterion(out.view(-1), batch.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader):.4f}")

Epoch 1, Loss: 0.6588
Epoch 2, Loss: 0.6600
Epoch 3, Loss: 0.6553
Epoch 4, Loss: 0.6546
Epoch 5, Loss: 0.6547
Epoch 6, Loss: 0.6536
Epoch 7, Loss: 0.6531
Epoch 8, Loss: 0.6533
Epoch 9, Loss: 0.6525
Epoch 10, Loss: 0.6519
Epoch 11, Loss: 0.6514
Epoch 12, Loss: 0.6484
Epoch 13, Loss: 0.6464
Epoch 14, Loss: 0.6478
Epoch 15, Loss: 0.6482
Epoch 16, Loss: 0.6471
Epoch 17, Loss: 0.6450
Epoch 18, Loss: 0.6442
Epoch 19, Loss: 0.6411
Epoch 20, Loss: 0.6418
Epoch 21, Loss: 0.6404
Epoch 22, Loss: 0.6415
Epoch 23, Loss: 0.6417
Epoch 24, Loss: 0.6389
Epoch 25, Loss: 0.6380
Epoch 26, Loss: 0.6398
Epoch 27, Loss: 0.6381
Epoch 28, Loss: 0.6403
Epoch 29, Loss: 0.6415
Epoch 30, Loss: 0.6385


In [35]:
# Evaluation
model.eval()
y_true, y_pred_prob = [], []
with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(device)
        out = model(batch.x, batch.edge_index, batch.batch)
        y_true.extend(batch.y.cpu().numpy())
        y_pred_prob.extend(out.view(-1).cpu().numpy())

In [57]:
# Best threshold by MCC
best_mcc, best_t = -1, None
for t in torch.arange(0.0, 1.0, 0.01):
    t = t.item()
    preds = (torch.tensor(y_pred_prob) >= t).int()
    mcc = matthews_corrcoef(y_true, preds)
    if mcc > best_mcc:
        best_mcc, best_t = mcc, t

y_pred = (torch.tensor(y_pred_prob) >= best_t).int()

print(f"Best Threshold: {best_t:.3f}")
print(f"AUC: {roc_auc_score(y_true, y_pred_prob):.4f}")
print(f"MCC: {best_mcc:.4f}")
print(f"Acc: {accuracy_score(y_true, y_pred):.4f}")
print(f"F1: {f1_score(y_true, y_pred):.4f}")

Best Threshold: 0.570
AUC: 0.6840
MCC: 0.3046
Acc: 0.6849
F1: 0.7589
